In [27]:
import pandas as pd
import plotly.express as px

from utils.backtest import Portfolio, Timeline
from utils.dummy import generate_stock_prices, get_quarterly_constituents, get_signals, sort_basket



In [28]:
stock_list = [f'STK{i:03d}' for i in range(1, 226)]
start_date = "2022-01-01"
end_date   = "2022-12-31"

prices = generate_stock_prices(stock_list, start_date, end_date)
print(f"Generated prices for {len(prices)} stocks from {start_date} to {end_date}.")

constituents = get_quarterly_constituents(stock_list, start_date, end_date)
print(f"Generated quarterly constituents for {len(constituents)} quarters.")

signals = get_signals(stock_list, start_date, end_date)
print(f"Generated signals and sorted baskets for {len(signals)} quarters.")


Generated prices for 225 stocks from 2022-01-01 to 2022-12-31.
Generated quarterly constituents for 4 quarters.
Generated signals and sorted baskets for 4 quarters.


In [29]:
def run_daily_backtest(
    stock_list,
    start_date,
    end_date,
    rebalance_every_days=21,
    asset_count=20,
    shorting_enabled=False,
):
    # get prices
    prices = generate_stock_prices(stock_list, start_date, end_date)
    price_df = pd.DataFrame(prices)
    
    # initialize
    dates = price_df.index
    initial_portfolio = Portfolio({}, {}, cash=1000.0)
    timeline = Timeline(dates[0], initial_portfolio)

    for i, date in enumerate(dates, start=1):
        # compute daily returns
        ret = price_df.loc[date] / price_df.shift(1).loc[date] - 1
        timeline.remark(date, ret)

        # rebalance periodically
        if i % rebalance_every_days == 0:
            basket = list(stock_list)
            sorted_basket, _ = sort_basket(basket)
            top = sorted_basket[:asset_count]
            bottom = sorted_basket[-asset_count:] if shorting_enabled else []
            timeline.rebalance(date, top, bottom)

    return timeline


In [30]:
tickers = [f'STK{i:03d}' for i in range(1, 226)]
tl = run_daily_backtest(tickers, '2022-01-01', '2022-12-31')
navs = tl.get_navs()
print(f"Final NAV: {navs.iloc[-1]:.2f}")
print(f"Annualized Return: {tl.calculate_annualized_return():.2%}")
print(f"Sharpe Ratio: {tl.calculate_sharpe_ratio():.2f}")

Final NAV: 1044.63
Annualized Return: 4.34%
Sharpe Ratio: 0.08


In [32]:
factor = "factor"
num_noise = 100

rebalance_days = 21
asset_count    = 20

# container for all timelines
timelines = {}

# run the “factor” backtest
timelines[factor] = run_daily_backtest(
    stock_list=[f"STK{i:03d}" for i in range(1,226)],
    start_date=start_date,
    end_date=end_date,
    rebalance_every_days=rebalance_days,
    asset_count=asset_count,
    shorting_enabled=False
)

# run N noise backtests
for i in range(num_noise):
    label = f"noise_{i}"
    timelines[label] = run_daily_backtest(
        stock_list=[f"STK{i:03d}" for i in range(1,226)],
        start_date=start_date,
        end_date=end_date,
        rebalance_every_days=rebalance_days,
        asset_count=asset_count,
        shorting_enabled=False
    )


In [33]:
factor_cr = pd.Series(
    timelines[factor].get_cumulative_returns(),
    name=factor
)

# noise series
noise_cr = [
    pd.Series(timelines[f"noise_{i}"].get_cumulative_returns(), name=f"noise_{i}")
    for i in range(num_noise)
]

df = pd.concat([factor_cr] + noise_cr, axis=1)

In [35]:
fig = px.line(
    df,
    y=[col for col in df.columns if col.startswith("noise_")],
    labels={"value": "Cumulative Return", "index": "Date"},
)

# style the noise traces
for trace in fig.data:
    trace.update(line=dict(color="black", width=2, dash="dot"),
                 opacity=0.1)

# overlay the factor line on top
fig.add_scatter(
    x=df.index,
    y=df[factor],
    mode="lines",
    name=factor,
    line=dict(color="red", width=4),
    marker=dict(size=7, color="red"),
)

fig.update_layout(
    width=1000,
    height=600,
    hovermode="closest",
    showlegend=False,
)

fig.show()

In [36]:
print(f"Factor: {factor}")
print(f"Annualized Return: ", timelines[factor].calculate_annualized_return())
print(f"Sharpe Ratio:", timelines[factor].calculate_sharpe_ratio())

Factor: factor
Annualized Return:  -0.018284980983955368
Sharpe Ratio: -0.032904806924876115


In [ ]:
# Tasks to do:
# Make sense of the generated code
# Use the Nikkei 225 data instead
# Use the Nikkei 225 index instead
# Use a valid signal like simple sentiment instead
# Use ELO score